# HW 3: Classification, Evaluation, and Deployment

In this homework, you will experience a complete machine learning cycle from data preparation to deployment. You will prepare/preprocess a dataset, make a model, evaluate models to find the best fit and deploy it to a simple web page. Our main objective is to make you try classification and evaluation methods, so we will only apply essential data preprocessing techniques but mainly focus on classification and evaluation.

We will use the **Adult** dataset from the UCI repository, and more information about the data is available [here](http://archive.ics.uci.edu/ml/datasets/Adult). Since we have removed and changed the dataset for grading purposes, use the one we provide on ilearn.

Based on census data, the dataset contains information to check whether income exceeds $50K/yr. The datasets consist of 14 attributes and one binary class variable:

- income: >50K, <=50K

- age: continuous.

- workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.

- fnlwgt: continuous.

- education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool. education-num: continuous.

- education-num: continuous.

- marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.

- occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.

- relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.

- race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.

- sex: Female, Male.

- capital-gain: continuous.

- capital-loss: continuous.

- hours-per-week: continuous.

- native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

and we have a binary class which can be `>50` or `<=50`.

**NOTE**
- Unlike the labs, each function you make here will be **graded**, so it is important to *strictly* follow the instruction.
- **Import** all necessary libraries yourself whenever needed. Failure to run any code can affect your grade.

## Contents

- **Total points: 6.0 pt**

0. Preparation (0.7 pt)
 - Task 1: Drop missing values (0.2 pt).
 - Task 2: Assign X and y (0.1 pt).
 - Task 3: One-hot encoding (0.1 pt).
 - Task 4: Train test split (0.1 pt).
 - Task 5: Standardization (0.2 pt).
1. Classification (2.9 pt)
 - Task 6: Random forest (0.5 pt).
 - Task 7: SVM with diverse kernels (0.4 pt).
 - Task 8: Decision tree and Random Forest (2.0 pt).
2. Evaluation (1.8 pt)
 - Task 9: Accuracy, Precision, Recall, F1-score (0.3 pt).
 - Task 10: AUC/AUPRC (0.3 pt).
 - Task 11: Apply them together with scikit-learn (0.4 pt).
 - Task 12: Manual implementation of performance metrics (0.8 pt).
3. Deployment (0.6 pt)
 - Task 13: Save models into a file using pickle (0.4 pt).
 - Task 14: DASH deployment (0.2 pt).

# 0. Preparation

##### Student information

Please provide your information for automatic grading.

In [1]:
STUD_SUID = 'khar1482'
STUD_NAME = 'Khondaker Refai Arafat'
STUD_EMAIL = 'khar1482@student.su.se'

#### Basic libraries

These libraries will be frequently used throughout the homework. Do not change the block below.

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score


RANDOM_STATE = 13579 #Do not change it!
np.random.seed(RANDOM_STATE) #Do not change it!

#### Load the dataset

Load the **Adult** dataset here using Pandas.

In [23]:
adult = pd.read_csv("datasets/adult.data", sep=",", header=None, skipinitialspace=True)

You can run the line below to give the dataframe proper column names.

In [24]:
adult.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race',  'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']

Here you can find out some basic information by calling *info(), head()*, and *describe()*.

In [5]:
adult.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [6]:
adult.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [7]:
adult.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


#### Task 1: Drop missing values (0.2 pt)

There is no null data, as we checked. However, if you read the dataset description, it says there are missing parts represented as "?". You can count them using the same technique we used for checking nulls in the previous lab. We have missing values in specific columns only, and it is about 5% of data records.

- Task 1-1: Count how many missing values (represented as "?" in our case) **each column** has and save it into the variable `missing_values` (0.1 pt).

In [8]:
## missing_values = None # CHANGE IT

missing_values = (adult == "?").sum()



In [9]:
print(missing_values)

age                  0
workclass         1836
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
income               0
dtype: int64


There are many ways to handle missing values, such as imputation or putting median/mean values, but we will practice the most straightforward way: removing the rows with missing values.

- Task 1-2: Complete the function below, removing any rows with missing values (0.1 pt).

In [10]:
def drop_missing_values(df, miss):
    """
    Input: 
      df: the dataframe (adult in our case)
      miss: a character to represent missing value ("?" in our case)
      
    Output: the dataframe without the missing values

    Step 1: Replace the value 'miss' with np.nan.
    Step 2: Drop the rows having nan values and store the result in data_dropped.
    Step 3: Return data_dropped
    
    """

    ##data_dropped = pd.DataFrame() # CHANGE IT
    ##return data_dropped
    
   
    df.replace(miss, np.nan, inplace=True)
    
   
    data_dropped = df.dropna()
    
    
    return data_dropped



- Apply `drop_missing_values` function to our dataset `adult` and save the result to `adult_dropped.` This part should be done correctly to get the point. You need to put our dataset and the indicator for missing values.

In [11]:
##adult_dropped = None # CHANGE IT

adult_dropped = drop_missing_values(adult, "?")
#adult_dropped.head()

- The output of the function should have the same attributes but only a smaller number of rows. Check how many rows are removed. Your dataset should have 30,162 rows!

In [12]:
adult_dropped.shape

(30162, 15)

#### Task 2: Assign X and y (0.1 pt)

Then let's split our dataset into two parts (`X` for attributes and `y` for labels) to use scikit-learn's various methods.
- Use `adult_dropped` from Task 1.
- `X` should have all the attributes without the labels (the last column).
- `y` should be a Pandas Series only with the labels.

In [13]:
##X = pd.DataFrame() # CHANGE IT!
##y = pd.Series(0) # CHANGE IT!

X = adult_dropped.drop('income', axis=1)

y = adult_dropped['income']


- Check the type and size here. We expect (30162, 14) for attributes (`X`) and (30162, ) for labels (`y`).

In [14]:
(X.shape, y.shape, type(X), type(y))

((30162, 14), (30162,), pandas.core.frame.DataFrame, pandas.core.series.Series)

#### Task 3: One-hot encoding (0.1 pt)

Unfortunately, scikit-learn does not support categorical attributes very well even for decision tree, and that means we need to convert them into reasonal form of numeric data to fit the algorithms. There is one way called one-hot encoding, which transforms the categorical data into multiple numeric columns for each possible value. There are various ways to apply this, especially using [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) or [Pandas](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html) but here we will use the Pandas function to keep the dataframe structure.

- Finish one_hot_encoding function which applies one-hot encoding to a given dataframe.


In [25]:
def one_hot_encoding(df):
    """
    Input:
        df: the attributes (X in our case)
    Output: one-hot encoded dataframe
    
    Step 1: Use pd.get_dummies to convert df to a one-hot-encoded form. 
            Enable an option called 'drop_first' to remove duplication.
    Step 2: Return the one-hot-encoded dataframe.
    
    * Those steps and suggested method are just for your convenience. You can use your own choice of methods.
      However, the result should be the same as the one created with the steps above.
    """
    
    #df_onehot = pd.DataFrame() # CHANGE IT
    #return df_onehot
    
    df_onehot = pd.get_dummies(df, drop_first=True)
    
    
    return df_onehot

- Create `X_onehot` by calling `one_hot_encoding` function with `X`. 

In [26]:
X_onehot = one_hot_encoding(X)

- Check your result by calling any methods you learned. If you successfully followed the instruction, the output (`X_onehot`) should have 96 columns.

In [27]:
X_onehot.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,39,77516,13,2174,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,83311,13,0,0,13,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,38,215646,9,0,0,40,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,53,234721,7,0,0,40,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,28,338409,13,0,0,40,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Task 4: Train test split (0.1 pt)

We also need to split our dataset further into four parts for evaluation.

- Use scikit-learn's `train_test_split` function to divide the dataset into four parts.
- Follow the instruction below carefully to get a point!.
    - Use `X_onehot` and `y`.
    - Assign 20% to a test set.
    - Use our random state (`RANDOM_STATE`)
    - Enable stratify option.

In [28]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = (pd.DataFrame(), pd.Series(0), pd.DataFrame(), pd.Series(0))
X_train, X_test, y_train, y_test = train_test_split(X_onehot, y, test_size=0.20, random_state=RANDOM_STATE, stratify=y)


- Check the type and size here. We expect 24,129 data instances in the training set.

In [20]:
(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

((24129, 96), (6033, 96), (24129,), (6033,))

#### Task 5: Standardization (0.2 pt)

After removing the missing value and splitting X and y, we need to take care of our numeric attributes. As you can check from `describe()` function, we have numeric attributes with different mean and standard deviation values. Not all machine learning models require standardization of numeric attributes, but some do. In this homework, SVM might be the case that the standardization is required. It might be better to make a standardized version when performing data preparation. 

- One-hot encoded data does not need to be standardized! So you need to choose the numeric columns only
- You need to import sklearn's `StandardScaler` for this task.

In [36]:


def standardize(X_train, X_test, numeric):
    """
    Input:
        - X_train: A split training set from Task 4
        - X_test: A split test set from Task 5
        - numeric: Numeric columns that should be standardized
    Output:
        - X_train_st: A standardized numeric attributes (ndarray)
        - X_test_st: A standardized numeric attributes (ndarray)

    Step 1: Initialize StandardScaler into the variable 'sc'.
    Step 2: Create X_train_numeric, X_test_numeric by selecting numeric columns from original X_train and X_test.
            Use the input 'numeric' to choose the columns.
    Step 3: Fit StandardScaler on X_train_numeric. You should only use the numeric columns only.
    Step 4: Use trained StandardScaler and run the transform function both on X_train_numeric (for the training set) 
            and X_test_numeric (for the test set). This job will standardize both training and test sets based on
            the statistics of training set. You should only use numeric attributes. Save the outputs to X_train_st and X_test_st.
    Step 5: Return X_train_st, X_test_st.
    
    """
    
    sc = StandardScaler()
    
    
    X_train_numeric = X_train[numeric]
    X_test_numeric = X_test[numeric]
    
    sc.fit(X_train_numeric)
    
    X_train_st = sc.transform(X_train_numeric)
    X_test_st = sc.transform(X_test_numeric)
    
    return X_train_st, X_test_st

    

In [37]:
def merge(X_train, X_test, X_train_numeric, X_test_numeric, numeric):
    # DO NOT CHANGE THIS FUNCTION
    # This function is to ensure that the datasets keep the Pandas DataFrame format.
    if X_train.shape == (0, 0): return pd.DataFrame([0]), pd.DataFrame([0])
    
    X_train_st_df = X_train.copy()
    X_train_st_df[numeric] = X_train_numeric
    X_test_st_df = X_test.copy()
    X_test_st_df[numeric] = X_test_numeric
    
    return X_train_st_df, X_test_st_df

- Find numeric columns first and assign the column names into the variable `numeric`. You can use `.info()` or `.describe()` function to find numeric columns.
- This `numeric` should contain the list of column names as strings, e.g., `['a', 'b', 'c']`.

In [40]:
##numeric = None # CHANGE IT
numeric = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
X_train.describe

<bound method NDFrame.describe of        age  fnlwgt  education-num  capital-gain  capital-loss  hours-per-week  \
15349   46  341762             13          7688             0              45   
32181   43  163434             13             0             0              55   
27666   44  162108             13             0             0              45   
19530   30  181091             13             0             0              40   
57      35   56352             11             0             0              40   
...    ...     ...            ...           ...           ...             ...   
20422   48  296066             10             0             0              50   
4661    45  162187             13             0             0              52   
4898    37  362062             15         99999             0              50   
16619   36  175360             14         13550             0              50   
18125   33   48702             11             0             0              

- Call `standardize` function to standardize numeric attributes. In this case, the output should only contain numeric attributes. We will merge the categorical features later on.

In [41]:
X_train_numeric, X_test_numeric = standardize(X_train, X_test, numeric)

- Check the mean and standard deviation values of the standardized dataset by running the blocks below. The dataset now should have near zero mean and one standard deviation.

In [42]:
X_train_numeric.mean(axis=0)

array([ 1.82575530e-16, -3.29813861e-17,  8.42203251e-17, -2.82697595e-17,
       -4.47604525e-17,  1.41348797e-17])

In [26]:
X_train_numeric.std(axis=0)

array([1., 1., 1., 1., 1., 1.])

- Unfortunately, scikit-learn's StandardScaler does not return DataFrame. Run the block below to recover DataFrame and categorical features.

In [43]:
X_train_st, X_test_st = merge(X_train, X_test, X_train_numeric, X_test_numeric, numeric)

- The final outcome (`X_train_st`) should have 96 columns again, where the numeric attributes have zero mean and one standard deviation.

In [45]:
X_train_st.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
count,2.412900e+04,2.412900e+04,2.412900e+04,2.412900e+04,2.412900e+04,2.412900e+04,24129.000000,24129.000000,24129.000000,24129.000000,...,24129.000000,24129.000000,24129.000000,24129.000000,24129.000000,24129.000000,24129.000000,24129.000000,24129.000000,24129.000000
mean,1.825755e-16,-3.298139e-17,8.422033e-17,-2.826976e-17,-4.476045e-17,1.413488e-17,0.070123,0.737577,0.035476,0.082888,...,0.001078,0.003689,0.000414,0.002445,0.001202,0.000663,0.000373,0.912512,0.002238,0.000497
std,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00,0.255360,0.439960,0.184983,0.275718,...,0.032809,0.060622,0.020354,0.049389,0.034648,0.025743,0.019310,0.282555,0.047255,0.022296
min,-1.637860e+00,-1.667683e+00,-3.574146e+00,-1.471472e-01,-2.153033e-01,-3.333406e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-7.991874e-01,-6.804452e-01,-4.392511e-01,-1.471472e-01,-2.153033e-01,-7.805663e-02,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,-1.130010e-01,-1.067059e-01,-4.738916e-02,-1.471472e-01,-2.153033e-01,-7.805663e-02,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,6.494285e-01,4.495231e-01,1.128197e+00,-1.471472e-01,-2.153033e-01,3.392959e-01,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,3.927875e+00,1.229248e+01,2.303782e+00,1.373371e+01,1.070320e+01,4.846703e+00,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


After finishing a simple data processing, let's proceed to our main task, classification.

# 1. Classification

In this assignment, we will run random forest (RF) and support vector machine (SVM) with different kernels using scikit-learn. Then, we will implement score functions for decision trees and main functions for random forests to understand the concepts better. We will continue to use the pre-processed Adult dataset from Tasks 1-5.

#### Task 6: Random forest (graded, 0.5 pt)

Here, you will run the random forest algorithm using scikit-learn and cross-validation. Detailed information about the random forest in scikit-learn can be found [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).

Your task is as follows:
 1. Create a random forest classifier `rf`.
     - Set the random state defined above (`RANDOM_STATE`).
     - Set the number of estimator to 100. 
     - Check the scikit-learn document to find the right parameter. 
 2. Report an average cross-validation score `rf_cross_val_score` with stratified k-fold with **cv=5**. You should report the average score, not a list of the scores. Use `X_onehot` and `y`, not the training or test set (0.2 pt). 
 - ***Note that you should report an average cross-validation score, which is one numeric value, not a list of scores.***


In [46]:
##rf = None # CHANGE IT

rf = RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=100)


In [47]:
##rf_cross_val_score = None # CHANGE IT

rf_cross_val_score = cross_val_score(rf, X_onehot, y, cv=5).mean()


- Run this line to check your score. Your score should be above 0.80.

In [48]:
rf_cross_val_score

0.85163467089276


 3. Run grid search `gs` with a single dictionary `grid_dict` with two keys.
    1) max_depth from 3 to 6 (included).
    2) min_samples_split = `[3, 5, 7]`.
4. Report the best classifier into the variable `rf_best_classifier`. 
   - Set **cv=5** for grid search cross-validation. Use our training set (`X_train_st` and `y_train`) to perform the grid search. 
   - This task can take a few minutes depending on computing power (0.3 pt).

In [49]:
##grid_dict = None # CHANGE IT

grid_dict = {
    'max_depth': [3, 4, 5, 6],
    'min_samples_split': [3, 5, 7]
}


In [50]:
##gs = None # CHANGE IT

gs = GridSearchCV(estimator=rf, param_grid=grid_dict, cv=5)
gs.fit(X_train_st, y_train)



GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=13579),
             param_grid={'max_depth': [3, 4, 5, 6],
                         'min_samples_split': [3, 5, 7]})

- Report your best classifier instance to `rf_best_classifier` (not the best score or average score).

In [51]:
##rf_best_classifier = None # CHANGE IT

rf_best_classifier = gs.best_estimator_


In [52]:
rf_best_classifier

RandomForestClassifier(max_depth=6, min_samples_split=5, random_state=13579)

#### Task 7: SVM with diverse kernels (graded, 0.4 pt)

We already tried a simple SVC with the RBF kernel before. Here you will run SVM again, but with different kernels and together with cross-validation. Detailed information about SVC in scikit-learn can be found [here](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC).

Your task is as follows:

  1. Create a standard SVC classifier `svc` without setting any parameter.
  2. Run a grid search with a list of two dictionaries, one with kernel = `['linear', 'poly', 'rbf']` and the other with C = `[1, 10, 100]`. 
     - This means you have to create a list containing two different dictionaries inside, not one dictionary with two keys. Each dictionary should only have one key.
  3. Report the best classifier into the variable `svm_best_classifier.` Set **cv=5** for grid search cross-validation. 
     - Use `X_train_st` and `y_train`. 
     - Depending on computing power (0.3 pt), this task can take a few minutes.


In [53]:
##svc = None # CHANGE IT

svc = SVC()


In [54]:
##grid_dict = None # CHANGE IT
##gs = None # CHANGE IT
##svm_best_classifier = None # CHANGE IT

grid_dict = [
    {'kernel': ['linear']},
    {'kernel': ['poly', 'rbf'], 'C': [1, 10, 100]}
]

gs = GridSearchCV(estimator=svc, param_grid=grid_dict, cv=5)
gs.fit(X_train_st, y_train)

svm_best_classifier = gs.best_estimator_


In [55]:
print(svm_best_classifier)

SVC(C=1)


  3. Take the best model from grid search and report the **test score** to `svm_gs_score` (0.1 pt).

In [56]:
##svm_gs_score = None # CHANGE IT

svm_gs_score = svm_best_classifier.score(X_test_st, y_test)


In [57]:
print(svm_gs_score)

0.8486656721365822


#### Task 8: Decision tree (2.0 pt)

**Task 8 involves manual implementation and uses a different dataset. If you would like to complete scikit-learn related tasks using the same dataset first, please move on to Task 9 and on and come back to Task 8 later.**

We will now implement a few modules for the decision tree. Follow the instructions carefully so that you can return a correct result. This task is composed of two sections as follows:

  - 8-1. Entropy, Gini index, and information gain (0.7 pt)
  - 8-2. Tree implementation (1.3 pt)

The first section of this task is to create three functions used to evaluate and grow the tree, which are covered in the lecture. Entropy and Gini index are the two main scores used for it. Information gain is the final score to choose a feature for dividing the node. Those scores are essential for a decision tree to work appropriately, and a wrong score can lead to choosing the features that are not proper for creating a high-performing tree.

- For simplicity, you will not use the **adult** dataset in this task but a simple **playgolf** dataset with categorical attributes.
- Task 8 is continuous, and the result of the function evaluates the grade. Since one function calls other functions in the task, failing to develop one function can affect the whole grade.

- Import the playgolf dataset to `playgolf` and check its properties. You can find it in the homework file.

In [58]:
playgolf = pd.read_csv('datasets/playgolf.csv')

In [59]:
playgolf.columns

Index(['Outlook', 'Temp', 'Humidity', 'Windy', 'Play Golf'], dtype='object')

In [60]:
playgolf.head()

,Outlook,Temp,Humidity,Windy,Play Golf
0,Rainy,Hot,High,False,No
1,Rainy,Hot,High,True,No
2,Overcast,Hot,High,False,Yes
3,Sunny,Mild,High,False,Yes
4,Sunny,Cool,Normal,False,Yes


- Task 8-1: Create a Gini index function (0.2 pt).
  - The Gini Index is calculated by subtracting the sum of the squared probabilities of each class from one.
  - You should double-check the lecture slides and the examples below to ensure you made a correct function.
  - You can use `collections.Counter()` to count labels of the dataset.
  - DO NOT USE SCIKIT-LEARN FOR THIS TASK!

In [61]:
from collections import Counter

def gini(dataset):
    """
    A function that calculates the Gini index of a given list.
    
    Input
     - dataset: a list of labels.
    Output
     - impurity: The Gini index of the list.
    
    You do not need to keep the output name of this function. The grade only depends on the correct outputs.
    
    """

    ##impurity = None # CHANGE IT
    ##return impurity
    
    total_count = len(dataset)
    
    label_counts = Counter(dataset)
    
    impurity = 1.0
    
    for label in label_counts:
        probability = label_counts[label] / total_count
        impurity -= probability**2
    
    return impurity

- Your Gini index is expected to have the following results:
  - `0.5` for `[0,0,1,1]`
  - `0.4082` for `[0,0,0,0,0,1,1]`

In [62]:
gini([0,0,1,1])

0.5

- Report the Gini score of the `Windy` attribute of **playgolf** to `gini_score` (0.2 pt).

In [63]:
##gini_score = None # CHANGE IT

gini_score = gini(playgolf['Windy'])


- Print your score here!

In [64]:
gini_score

0.48979591836734704

- Task 8-2: Create an entropy function (0.2 pt).
  - You should double check the lecture slides and the examples below to make sure you made a correct function. 
  - You can use `collections.Counter()` to count labels of the dataset.
  - DO NOT USE SCIKIT-LEARN FOR THIS TASK!

In [65]:
from collections import Counter
import math

def entropy(dataset):
    """
    A function that calculates the entropy of a given list.
    
    Input
     - dataset: a list of labels.
    Output
     - impurity: entropy value of the list.
    
    You do not need to keep the output name of this function. The grade only depends on the correct outputs.
    
    """

    ##impurity = None # CHANGE IT
    ##return impurity
    
    total_count = len(dataset)
    
    label_counts = Counter(dataset)
    
    impurity = 0.0
    
    for label in label_counts:
        probability = label_counts[label] / total_count
        impurity -= probability * math.log2(probability)
    
    return impurity

- Your entropy is expected to have the following results:
  - `1.0` for `[0,0,1,1]`
  - `0.8631` for `[0,0,0,0,0,1,1]`

In [66]:
entropy([0,0,1,1])

1.0

In [67]:
entropy([0,0,0,0,0,1,1])

0.863120568566631

- Report the entropy score of the `Temp` attribute of **playgolf** to `entropy_score` (0.2 pt).

In [68]:
##entropy_score = None # CHANGE IT

entropy_score = entropy(playgolf['Temp'])


- Print your score here!

In [69]:
entropy_score

1.5566567074628228

- Task 8-3: Create an information gain function (0.3 pt). 
  - **DO NOT use entropy but only use the Gini index for scores.**
  - Check the lecture slides and the examples below to ensure you made a correct function.

In [70]:
from collections import Counter

def gini(labels):
    total_count = len(labels)
    label_counts = Counter(labels)
    gini_index = 1.0
    for label in label_counts:
        probability = label_counts[label] / total_count
        gini_index -= probability**2
    return gini_index


def information_gain(labels_start, labels_split):
    """
    Calculate information gain when we have information on label distribution before and after the split operation.
    This information gain function receives two values:
    
    Input:
      - labels_start: A single list of all current labels
        e.g.) [0,0,0,0,1,1,1,1]
      - labels_split: A list of lists representing the split 
        e.g.) [ [0,0,1,1], [1,1,0,0] ]
    
    Then, we can calculate information gain by calculating the Gini index before splitting,
    and subtract (Gini index of the subset * proportion of the subset) for each list after splitting from there.
    
    Output:
      - info_gain: Information gain
    
    You do not need to keep the output name of this function. The grade only depends on the correct outputs.
    
    """

    ##info_gain = None # CHANGE IT
    ##return info_gain
    
    gini_start = gini(labels_start)
    gini_split = 0.0
    total_count = len(labels_start)
    for subset in labels_split:
        subset_count = len(subset)
        gini_split += (subset_count / total_count) * gini(subset)
    info_gain = gini_start - gini_split
    return info_gain


In [71]:
information_gain([0,0,0,0,1,1,1,1], [[0,0,1,0],[1,1,0,1]])

0.125

- Your information gain is expected to have the following results:
  - `0.0` for `[0,0,0,0,1,1,1,1], [[0,0,1,1],[0,0,1,1]]`
  - `0.5` for `[0,0,0,0,1,1,1,1], [[0,0,0,0],[1,1,1,1]]`
  - `0.125` for `[0,0,0,0,1,1,1,1], [[0,0,1,0],[1,0,1,1]]`

- Here we have labels before and after splitting information. Use those two values to calculate information gain and report it to `info_gain_score` using your own `information_gain` function (0.3 pt). Make sure that your infomation gain function support categorical labels not only binary values!

In [72]:
labels_start = [1,2,1,2,2,1,2,1,3,3,3]
labels_split = [[3,3,3],[1,2,1,1],[2,2,1,2]]

In [73]:
info_gain_score = information_gain(labels_start, labels_split)

- Print your score here!

In [74]:
info_gain_score

0.38842975206611574

Now, we have some score functions required to construct the tree. Then, it is time to create the tree itself! 

- Note that this assignment does not aim to make whole working decision trees and random forests but the core functions to understand the algorithm.

 The tree works as follows:

- Starting from the root, you choose a few attributes to test. This does not need to be all the attributes the dataset has.
- Iterate chosen attributes and calculate information gain, assuming you split the dataset based on each attribute.
- Choose the column (attribute) with maximum information gain and split the dataset again.
- Continue growing the tree by choosing the column in the same way until we meet a closing criterion.

Here, we will give you a basic `split` function used to split the dataset based on the attribute. This split function receives the attributes (`X`), the label (`y`), and one `feature` (attribute) of it, and splits the whole dataset based on the categories of the selected feature and returns split data subsets and label subsets. You will use those split values to make a few functions needed for decision trees and random forests. 

In [75]:
def split(X, y, attr):
    split_attrs = []
    split_labels = []
    
    for val in X[attr].unique():
        attr_subset = []
        label_subset = []
        
        for idx, row in X.iterrows():
            
            if row[attr] == val:
                attr_subset.append(row)
                label_subset.append(y[idx])
                
        split_attrs.append(pd.DataFrame(attr_subset))
        split_labels.append(label_subset)
        
    return split_attrs, split_labels

Check out the result by running the function below, and also check the `Windy` column to understand what the function does.

In [76]:
split(playgolf.drop('Play Golf', axis=1), playgolf['Play Golf'], 'Windy')

([     Outlook  Temp Humidity  Windy
  0      Rainy   Hot     High  False
  2   Overcast   Hot     High  False
  3      Sunny  Mild     High  False
  4      Sunny  Cool   Normal  False
  7      Rainy  Mild     High  False
  8      Rainy  Cool   Normal  False
  9      Sunny  Mild   Normal  False
  12  Overcast   Hot   Normal  False,
       Outlook  Temp Humidity  Windy
  1      Rainy   Hot     High   True
  5      Sunny  Cool   Normal   True
  6   Overcast  Cool   Normal   True
  10     Rainy  Mild   Normal   True
  11  Overcast  Mild     High   True
  13     Sunny  Mild     High   True],
 [['No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes'],
  ['No', 'No', 'Yes', 'Yes', 'Yes', 'No']])

In [77]:
playgolf['Windy']

0     False
1      True
2     False
3     False
4     False
5      True
6      True
7     False
8     False
9     False
10     True
11     True
12    False
13     True
Name: Windy, dtype: bool

- Task 8-4: The first thing we need to make is the function to choose the attributes. So we receive a dataframe and a strategy and return a list of chosen columns. We can have three different options for selecting attributes (0.3 pt).
  - `sqrt`: use the sqrt of total column size to choose the columns
  - integer numbers: use the received number for  
  - `max`: choose the same number of columns as its original size - return the columns.

In [78]:
def select_attributes(X, strategy):
    """
    Input
        - X: Attributes of the node.
        - strategy: a strategy for the number of attributes the algorithm chooses.
    Output
        - attributes: a list of selected attributes

    Step 1: Check the strategy. If the type of strategy is integer, the number of attributes to choose will be that number.
            If it's "sqrt", then it will be the square root of column size (if it is a floating point number, round it down). 
            If "max", it's the size of the dataset's columns. Put appropriate value to 'num_attr'.
    Step 2: Choose 'num_attr' column names from X.columns "without" allowing replacement. You can use np.random.choice or equivalent. 
            Assign the result to 'attributes'.
    Step 3: return 'attributes'.

    """
    
    ##num_attr = None # CHANGE IT
    ##attributes = None # CHANGE IT
        
    ##return None # CHANGE IT
    
    if isinstance(strategy, int):
        num_attr = strategy
    elif strategy == "sqrt":
        num_attr = int(np.sqrt(X.shape[1]))
    elif strategy == "max":
        num_attr = X.shape[1]
    else:
        raise ValueError("Invalid strategy. Choose an integer, 'sqrt', or 'max'.")

    attributes = np.random.choice(X.columns, num_attr, replace=False)
    return attributes



- You can test your method here!

In [79]:
select_attributes(playgolf.drop('Play Golf', axis=1), 2)

array(['Temp', 'Outlook'], dtype=object)

In [80]:
select_attributes(playgolf.drop('Play Golf', axis=1), "max")

array(['Humidity', 'Outlook', 'Temp', 'Windy'], dtype=object)

- Task 8-5: Now we have a method to choose attributes. Then now we need to make a function to iterate those selected attributes in the dataset and eventually can make a function for choosing the best feature to split, given the dataset of the node (It will be a full dataset if we run this function on the root node.). The function receives the datasets (`X`, `y`) and the list of selected attributes calculated by `selected_attributes` function (or it can also be user input), and returns the best feature among the chosen one and it's information gain. This process is one of the core processes of the decision tree (0.3 pt). 

In [81]:

def check_info_gain_per_attribute(X, y, attributes):
    """
    Input
        - X: Attributes of the node.
        - y: dataset labels.
        - attributes: the selected attributes to test.
    Output
        - best_feature: The best feature in terms of information gain.
        - best_gain: The information gain value when the dataset is split by the best feature.
        
    Step 1: Initialize two variables: Set best_info_gain to zero and best_attr to None.
    Step 2: You should iterate the attributes we get as input.
            For each chosen attribute, 'split' the dataset using the split function we have offered.
            This will return sets of attributes and labels. Save the split attributes and labels.
    Step 3: Calculate the information gain of the current split in the iteration. 
            Use the information_gain function you created and the label information from Step 3.
    Step 4: Compare it to the current best gain. If the new gain is higher (not higher or equal to), reset best_gain and best_feature.
    Step 5: Return best_attr, best_info_gain.
    
    
    """

    
    
    best_info_gain = 0.0
    best_attr = None

    for attr in attributes:
        
        subset_labels = [y[X.index[X[attr] == value]] for value in X[attr].unique()]
        info_gain = information_gain(y, subset_labels)

        
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_attr = attr

    
    return best_attr, best_info_gain


- Now we can check and you need to answer here! - which attribute is better in terms of information gain between `Windy` and `Outlook`?
  - Answering this question is also part of Task 8-5.
  - You should not put the string value yourself. Save the outcome of `check_info_gain_per_attribute` function that answers the question.

In [82]:
##best_attr, better_attribute = None, None # CHANGE IT

best_attr, better_attribute = check_info_gain_per_attribute(playgolf.drop('Play Golf', axis=1), playgolf['Play Golf'], ['Windy', 'Outlook'])

In [83]:
print(better_attribute)

0.11632653061224485


Task 8-6: Now we can create a wrapper function that calls two separate function: `select_attributes`, and `check_info_gain_per_attribute`. Now, the function receives data `X`, `y`, and `strategy` and calls `select_attributes` with `strategy` to get a list of attributes. Next it calls `check_info_gain_per_attribute` with selected attributes and finally return the best attribute and the corresponding information gain (0.2 pt).

In [84]:
# 0.2 pt
def best_split(X, y, strategy):
    """
    Input
        - X: Attributes of the node.
        - y: dataset labels.
        - strategy: a strategy for the number of attributes the algorithm chooses.
    Output
        - best_feature: The best feature in terms of information gain.
        - best_gain: The information gain value when the dataset is split by the best feature.

    Complete the function following the instructions above
    """
    ##best_attr, best_info_gain = None, None # CHANGE IT
    
    ##return best_attr, best_info_gain
    
    
    selected_attributes = select_attributes(X, strategy)
    
    
    best_attr, best_info_gain = check_info_gain_per_attribute(X, y, selected_attributes)
    
    return best_attr, best_info_gain


- Find the best split of the playgolf dataset with `strategy` = `sqrt`. Report the best feature and best gain to `best_attr_playgolf` and `best_gain_playgolf`.
   - Answering the question here is part of Task 8-6

In [85]:
np.random.seed(RANDOM_STATE)
##best_attr_playgolf, best_gain_playgolf = None, None # CHANGE IT

best_attr_playgolf, best_gain_playgolf = best_split(playgolf.drop('Play Golf', axis=1), playgolf['Play Golf'], "sqrt")



In [86]:
# TEST YOUR RESULT HERE
best_attr_playgolf, best_gain_playgolf

('Windy', 0.030612244897959162)

Task 8-7: Now we have functions 1) to split the function based on one chosen feature (`split`) and 2) to choose the best feature to split (`best_split`). The next step will be to create one tree with all the information we have. Finish the function `build`. This function makes one tree of the random forest by using two previous functions. Since this function is a recursive one, it will return a complete tree, not a node (0.5 pt).

In [87]:
# 0.5 pt

def build(X, y, strategy, max_depth = 5, min_samples_leaf = 5, tol=0.00001, _depth = 0):
    """
    Input
        - X: Attributes of the data
        - y: dataset labels
        - strategy: a strategy for the number of attributes the algorithm chooses.
        - max_depth: maximum allowed depth of the tree
        - min_samples_leaf: minimum number of data instances required to continue
        - tol: information gain tolerance value.
        - _depth: current depth of tree starting from zero (root). Only controlled by the algorithm.
    Output
        - node: a leaf or middle node.
    
    Step 0: Consider some stopping criteria. We do not continue this function if ONE of the following conditions is met:
        1. if the current depth number is bigger than max_depth
        2. if the current sample size is smaller than min_samples_leaf
      Check these conditions and terminate the function if required. When terminating, return {"type": "leaf", "majority": the most common class label (y)}.
    Step 1: Run the best split function to get the best attributes and the best information gain for the node.
    Step 2: Examine the best information gain value. If it is lower than the tolerance value (tol), 
            return the node with the best information gain value. The node should be a dictionary form 
            {"type": "leaf", "gain": the best information gain, "majority": the most common class label (y)}.
    Step 3: If the best information gain is higher, split the dataset with the chosen best attribute.
    Step 4: Create an empty list called "branches" to save all the branches of the current node. 
            This branches list will contain the returned values of the recursive calls of the build function.
    Step 5: Run this 'build' function recursively for each split attribute and label and store the result
            to the  "branches" list **only if the returned value is not False (from termination)**.
            Remember to increase the depth value so we can trace the max_depth.
            Note that depending on your implementation your indices for X and y may mismatch. You need to always make sure that your featurea and labels are correctly ordered.
    Step 6: After all the recursion process is done, return the root node with its best attribute, branch information (i.e., "branches" list),
            and the best information gain (fill in the right variables into the dictionary!).
    """
    

    
    if _depth >= max_depth or len(y) < min_samples_leaf:
        majority_class = y.mode().iloc[0] if len(y) > 0 else None
        return {"type": "leaf", "majority": majority_class}
    
    
    best_attr, best_info_gain = best_split(X, y, strategy)
    
    
    if best_info_gain < tol:
        majority_class = y.mode().iloc[0]
        return {"type": "leaf", "gain": best_info_gain, "majority": majority_class}
    
    
    split_attrs, split_labels = split(X, y, best_attr)
    
    
    branches = []
    
    
    for label, sub_X, sub_y in zip(split_attrs.keys(), split_attrs.values(), split_labels.values()):
        result = build(sub_X, sub_y, strategy, max_depth, min_samples_leaf, tol, _depth + 1)
        if result is not None:
            branches.append({"label": label, "child": result})
    
    
    return {
        "type": "node",
        "best_feature": best_attr,
        "branches": branches,
        "value": best_info_gain
    }


- Bulid one tree using the whole playgolf dataset and report the tree to the variable 'single_tree'. Set the strategy to "sqrt", max_depth to 3, and min_samples_leaf to 2. You do not need to specify the tolerance value (0.5 pt).

In [88]:


"""
from sklearn.tree import DecisionTreeClassifier

# Define the parameters
strategy = "sqrt"
max_depth = 3
min_samples_leaf = 2

# Create a Decision Tree Classifier with the specified parameters
single_tree = DecisionTreeClassifier(criterion='entropy', splitter=strategy, max_depth=max_depth, min_samples_leaf=min_samples_leaf)

# Fit the tree to the data
single_tree.fit(X, y)

"""
from sklearn.tree import DecisionTreeClassifier

playgolf = pd.read_csv('datasets/playgolf.csv')
np.random.seed(RANDOM_STATE)
strategy = "sqrt"
max_depth = 3
min_samples_leaf = 2
X = playgolf.drop('Play Golf', axis=1).values
y = playgolf['Play Golf'].values

def build(X, y, max_depth, min_samples_leaf):
    if max_depth <= 0 or len(y) <= min_samples_leaf:
        return np.bincount(y).argmax()
    best_feature = None
    left_mask = None
    right_mask = None 
    
    left_subtree = build(X[left_mask], y[left_mask], max_depth - 1, min_samples_leaf)
    right_subtree = build(X[right_mask], y[right_mask], max_depth - 1, min_samples_leaf)
    return (best_feature, left_subtree, right_subtree)
single_tree = DecisionTreeClassifier(criterion='entropy', splitter='best', max_depth=max_depth, min_samples_leaf=min_samples_leaf)




- Print your result here

In [89]:
single_tree

DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=2)

# 2. Evaluation 

#### Task 9: Accuracy, Precision, Recall, F1-score (0.3 pt)

We start to use our original dataset again! Remember that we have the standardized datasets (`X_train_st` and `X_test_st`). You will evaluate the random forest and the support vector machine classifier with various performance measures you have learned besides accuracy, such as precision, recall, and F1-score, also using scikit-learn. Here, we continue to use the same dataset.

- Throughout the task, use standardized datasets (`X_train_st`, `X_test_st`). To use the various score functions here, you need to convert the labels of `y_train` and `y_test` (`<=50K`, `>50K`) to numeric ones (0 or 1) since the score functions will not recognize the categorical labels. Create `y_train_numeric` and `y_test_numeric` with the converted labels (`<=50K` to 0, and `>50K` to 1). Refer to the previous labs. (0.1 pt)

In [90]:
# Subtask 1: 0.1 pt
##y_train_numeric = pd.Series(0) # CHANGE IT
##y_test_numeric = pd.Series(0) # CHANGE IT

from sklearn.preprocessing import LabelEncoder


import pandas as pd


label_mapping = {'<=50K': 0, '>50K': 1}

y_train_numeric = y_train.map(label_mapping)

y_test_numeric = y_test.map(label_mapping)

Check if you successfully replaced the values here.

In [91]:
y_train_numeric.unique(), y_test_numeric.unique()

(array([1, 0], dtype=int64), array([0, 1], dtype=int64))

- Create an instance of an SVC classifier with the **polynomial** kernel and fit the model using the training set. You can use any variable name for your SVC instance. Report the precision score, recall score, and F1-score on the test set, and save it into the variable `precision_score_svc`, `recall_score_svc`, and `f1_score_svc` (0.2 pt). 
  - You can find out the information about the performance measures [here](https://scikit-learn.org/stable/modules/model_evaluation.html#precision-recall-f-measure-metrics). We require you to calculate the scores using the following functions in scikit-learn: `precision_score`, `recall_score`, and `f1_score`. 
  - Use the `macro` average option - you can read more about it [here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html).
  - There is no partial point if you are correct on only some of them.

In [92]:
# Subtask 2: 0.2 pt
##precision_score_svc = None # CHANGE IT
##recall_score_svc = None # CHANGE IT
##f1_score_svc = None # CHANGE IT

from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score


svc_classifier = SVC(kernel='poly', gamma='auto')  

svc_classifier.fit(X_train_st, y_train_numeric)

y_pred_svc = svc_classifier.predict(X_test_st)

precision_score_svc = precision_score(y_test_numeric, y_pred_svc, average='macro')
recall_score_svc = recall_score(y_test_numeric, y_pred_svc, average='macro')
f1_score_svc = f1_score(y_test_numeric, y_pred_svc, average='macro')

precision_score_svc, recall_score_svc, f1_score_svc


(0.8381345194851315, 0.5660371619566467, 0.555194066367671)

Print three scores here.

In [93]:
precision_score_svc, recall_score_svc, f1_score_svc

(0.8381345194851315, 0.5660371619566467, 0.555194066367671)

#### Task 10: AUC / AUPRC (0.3 pt)

You will evaluate the random forest and the support vector machine classifier with various performance measures related to the ROC curve, such as the area under the ROC curve (AUC) and the area under the precision-recall curve (AUPRC). Use the same dataset as the ones for Task 7 (`X_train_st` and `y_train_numeric`). AUPRC and AUC scores also only recognize numeric labels.

- Create an instance of a random forest classifier without setting any constraint. Don't forget to set the random state to our value `RANDOM_STATE` and fit the model on the training set.

In [94]:
##rf_10 = None # CHANGE IT
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score

rf_10 = RandomForestClassifier(random_state=RANDOM_STATE)

rf_10.fit(X_train_st, y_train_numeric)


RandomForestClassifier(random_state=13579)

- Print the accuracy on the test set to `accuracy_rf` (0.1 pt).

In [95]:
##accuracy_rf = None # CHANGE IT

accuracy_rf = accuracy_score(y_test_numeric, rf_10.predict(X_test_st))




- Report AUC and AUPRC using the test set and save it into the variable *auc_rf, auprc_rf*. You can find out the information about the performance measures [here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score) for AUC score, and [here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html) for AUPRC score (0.2 pt). 
  - AUPRC has many names and is supported as *average precision score* in scikit-learn. We require you to calculate the scores using the following functions: *roc_auc_score, average_precision_score*. If you are correct on only some of them, there is no partial point.
  - You should use the probability of the predictions to calculate AUC/AUPRC.
  - Use the `weighted` average option.

In [96]:
##auc_rf = None # CHANGE IT
##auprc_rf = None # CHANGE IT

from sklearn.metrics import roc_auc_score, average_precision_score

y_prob_rf = rf_10.predict_proba(X_test_st)[:, 1]

auc_rf = roc_auc_score(y_test_numeric, y_prob_rf, average='weighted')

auprc_rf = average_precision_score(y_test_numeric, y_prob_rf, average='weighted')




Print your scores here!

In [97]:
(accuracy_rf, auc_rf, auprc_rf)


(0.8483341621084037, 0.9001796618706874, 0.7680117192095133)

#### Task 11: Apply them together with scikit-learn (0.4 pt)

Here, you will try to apply the grid search using the performance measures you have tried on Task 5 and Task 6 and pick the best-performing model in terms of specific performance measures.

Our dataset is imbalanced, meaning that the healthy patient is dominant. Therefore, we can expect that the best model can be different, and we may also need to use AUPRC to get the most suitable model. 

Your task is as follows:

1. Use the same dataset as the ones for Task 7 (`X_train_st` and `y_train_numeric`). AUPRC and precision scores also only recognize numeric labels.
2. Create an instance of a kNN classifier without setting any constraints. 
3. Run grid search with a dictionary - here, you need to search the number of neighbors for kNN from 1 to 10 (included) and use two different scoring measures: AUPRC and precision. Set `cv=3` for grid search cross-validation. **Do not specify any average option.**
4. Put the best classifiers into the respective variable called `auprc_best_classifier` and `precision_best_classifier`. Note that we would like to get **two** best classifiers per evaluation score.

If you complete the method, you can run the following line to check whether your functions are correct. We will evaluate your functions with different data, so implement them carefully.

In [122]:
##auprc_best_classifier = None # CHANGE IT
##precision_best_classifier = None # CHANGE IT

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, average_precision_score, precision_score

knn_classifier = KNeighborsClassifier()

param_grid = {'n_neighbors': list(range(1, 11))}

auprc_scorer = make_scorer(average_precision_score, greater_is_better=True)
precision_scorer = make_scorer(precision_score, greater_is_better=True)

grid_auprc = GridSearchCV(knn_classifier, param_grid, cv=3, scoring=auprc_scorer)
grid_precision = GridSearchCV(knn_classifier, param_grid, cv=3, scoring=precision_scorer)

grid_auprc.fit(X_train_st, y_train_numeric)
grid_precision.fit(X_train_st, y_train_numeric)

auprc_best_classifier = grid_auprc.best_estimator_
precision_best_classifier = grid_precision.best_estimator_




c:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 353, in _score
    y_pred = method_caller(estimator, "predict", X)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py

Check your best classifiers here!

In [121]:
(auprc_best_classifier, precision_best_classifier)

(KNeighborsClassifier(n_neighbors=1), KNeighborsClassifier(n_neighbors=1))

#### Task 12: Task 5 implementation (0.8 pt)

This task requires you to implement the following performance measures:
 - Accuracy (0.2 pt)
 - Precision (0.2 pt)
 - Recall (0.2 pt)
 - F1-score (0.2 pt)
 
All inputs will be the NumPy arrays, so you can use any NumPy array methods to calculate the scores.
  - **Scikit-learn methods are not allowed**
  - Looping the list is not allowed
  - Copying materials from the internet is completely prohibited

We want you to calculate the **micro** average for Precision and Recall. Read more about it [here](https://tomaxent.com/2018/04/27/Micro-and-Macro-average-of-Precision-Recall-and-F-Score/). You can always validate your method by comparing the result to the one from scikit-learn.

We only test these functions with binary values so you do not need to handle categorical values.

In [100]:

def accuracy_manual(truth, predicted):
    ##return None # CHANGE IT
    return np.mean(truth == predicted)

In [101]:
def precision_manual(truth, predicted):
    ##return None # CHANGE IT
    true_positives = np.sum((predicted == 1) & (truth == 1))
    false_positives = np.sum((predicted == 1) & (truth == 0))
    return true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0.0


In [102]:
def recall_manual(truth, predicted):
    ##return None # CHANGE IT
    true_positives = np.sum((predicted == 1) & (truth == 1))
    false_negatives = np.sum((predicted == 0) & (truth == 1))
    return true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0.0


In [103]:
def f1_manual(truth, predicted):
    ##return None # CHANGE IT
    precision = precision_manual(truth, predicted)
    recall = recall_manual(truth, predicted)
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0


Assign the results of your four function on two arrays (`truth`, `predicted`).

In [104]:
truth     = np.array([0,1,0,1,1,1,1,0,0,1,1,0,0,1,1,0,1])
predicted = np.array([1,0,0,0,1,0,0,1,1,0,1,1,1,0,1,1,1])

In [105]:
##accuracy_score_manual = None # CHANGE IT
##precision_score_manual = None # CHANGE IT
##recall_score_manual = None # CHANGE IT
##f1_score_manual = None # CHANGE IT

accuracy_score_manual = accuracy_manual(truth, predicted)
precision_score_manual = precision_manual(truth, predicted)
recall_score_manual = recall_manual(truth, predicted)
f1_score_manual = f1_manual(truth, predicted)


Show your results here!

In [106]:
(accuracy_score_manual, precision_score_manual, recall_score_manual, f1_score_manual)

(0.29411764705882354, 0.4, 0.4, 0.4000000000000001)

# 3. Deployment

**Task 13: Save models into a file using pickle (0.4 pt)**

In this task, you need to pick the best model using cross-validation and deploy it as a pickle file. For this task, we will use the diabetes data that we used for Homework 1. This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. The objective of the dataset is to diagnostically predict whether or not a patient has diabetes, based on certain diagnostic measurements included in the dataset. Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage. You can find it in the homework folder.

- Load the dataset into `diabetes`.

In [107]:
diabetes = pd.read_csv("datasets/diabetes.csv")

In [108]:
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


- Split the dataset into two parts: attributes (`X`) and labels (Outcome, `y`).

In [109]:
##X = None # CHANGE IT
##y = None # CHANGE IT

import pandas as pd
from sklearn.model_selection import train_test_split

X = diabetes.drop("Outcome", axis=1)
y = diabetes["Outcome"]


Your task is as follows:

1. Use scikit-learn's `train_test_split` function to divide the dataset into four parts.
- Follow the instruction below carefully to get a point!.
    - Use `X` and `y`.
    - Assign 10% to a test set.
    - Use our random state (`RANDOM_STATE`)
    - Enable stratify option.


In [110]:
# Remove the assigned values and write the train_test_split function
##X_train, X_test, y_train, y_test = (pd.DataFrame(), pd.Series(0), pd.DataFrame(), pd.Series(0))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=RANDOM_STATE, stratify=y)

2. Create an instance of an SVC classifier without setting any constraint.
3. Run grid search with a list of two dictionaries. 
    - In the first dictionary, you should examine 'poly' kernel, with degree `[3, 4 5]`. 
    - In the second dictionary, you should test two kernels `['linear', 'rbf']` with a list of C values `[5, 10, 100]`. 
    - Use **AUPRC** as its scoring measure. Set `cv=3` for grid search cross-validation. 
    - Use the training set to train the grid search instance.
4. Save the best classifier into the variable called `svm_best_classifier_dash` and save the trained model into `model_diabetes.pickle` using pickle. When saving your model, do not specify any folder.
 - **Do not use your own specific name for the model!**

In [111]:
##svm_best_classifier_dash = None # CHANGE IT
from sklearn.svm import SVC

svm_classifier = SVC()


- Show your classifier here!

In [112]:
##svm_best_classifier_dash

from sklearn.model_selection import GridSearchCV

param_grid = [
    {'kernel': ['poly'], 'degree': [3, 4, 5]},
    {'kernel': ['linear', 'rbf'], 'C': [5, 10, 100]}
]

grid_search = GridSearchCV(svm_classifier, param_grid, cv=3, scoring='average_precision', n_jobs=-1)
grid_search.fit(X_train, y_train)


GridSearchCV(cv=3, estimator=SVC(), n_jobs=-1,
             param_grid=[{'degree': [3, 4, 5], 'kernel': ['poly']},
                         {'C': [5, 10, 100], 'kernel': ['linear', 'rbf']}],
             scoring='average_precision')

5. Using the best classifier, report the **test** score to `svm_test_score`.

In [114]:
##svm_test_score = None # CHANGE IT
import pickle

svm_best_classifier_dash = grid_search.best_estimator_
with open("model_diabetes.pickle", "wb") as model_file:
    pickle.dump(svm_best_classifier_dash, model_file)

svm_test_score = svm_best_classifier_dash.score(X_test, y_test)

(svm_test_score)

0.7142857142857143

**Task 14: DASH deployment (0.2 pt)**

You will run DASH application by using the model `model_diabetes.pickle` you exported with the project files in the `webplatform_dash` folder. Locate the model in the same folder with this jupeter notebook, and go into `webplatform_dash` folder. There you can run your own DASH application as you learned from Lab 5. Note that you should **not** move the model file into `webplatform_dash`.

- Submit a screenshot with your model file into one zip file in a separate submission form for your DASH project. For the details of the DASH deployment, check out Lab 5.